# Figure 8

## Imports

In [ ]:
import dolfin  
import matplotlib.pyplot as plt
import numpy
import sympy
import sys

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

from matplotlib.collections import PatchCollection

## Defining Geometry and material parameters

In [ ]:
seeds_filename = "Fig8-seeds.dat"
mesh_filebasename = "Fig8-mesh"

domain_y = 1
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = False)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)
Es = 1
nus = 0.2
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])
vol = (xmax - xmin)*(ymax - ymin)
dV = dolfin.Measure("dx",domain=mesh)
Phi_s0 = dolfin.assemble(1*dV)/vol

vertices = numpy.array([[xmin, ymin],
                     [xmax, ymin],
                     [xmax, ymax],
                     [xmin, ymax]])
bbox = [xmin, xmax, ymin, ymax]

## Kinematics

In [ ]:

alpha_x, alpha_y, alpha_xy, alpha_yx = sympy.symbols('alpha_x alpha_y alpha_xy alpha_yx')
epsilon_x = sympy.symbols('epsilon_x')
epsilon_y = sympy.symbols('epsilon_y')
epsilon_xy = sympy.symbols('epsilon_xy')
epsilon = sympy.symbols('epsilon')
C_CM = sympy.MatrixSymbol('C', 2, 2).as_explicit()

C00, C01, C10, C11 = sympy.symbols('C00 C01 C10 C11')
C_CM = sympy.Matrix([[C00, C01],
                  [C10, C11]])


p = sympy.symbols('p')
I_C_CM   = sympy.trace(C_CM) 
II_C_CM  = (sympy.trace(C_CM)**2 - sympy.trace(C_CM**2))/2 
III_C_CM = sympy.det(C_CM) 
J_CM     = sympy.sqrt(III_C_CM)
E_CM = (C_CM - sympy.eye(2))/2 

C_CM_bar = J_CM**(-2/3)*C_CM
I_C_CM_bar   = sympy.trace(C_CM_bar) 
II_C_CM_bar  = (sympy.trace(C_CM_bar)**2 - sympy.trace(C_CM_bar**2))/2 


F = sympy.Matrix(
        [[alpha_x,    0  ],\
        [   0   , alpha_y]])

J = F.det()
C = F.T * F
E = (C - sympy.eye(2))/2 

## Macroscopic model

In [ ]:
def macroscopic_model(mat_params):
    homo = dmech.HomogenizationProblem(
        dim=2,
        mesh=mesh,
        mat_params=mat_params["parameters"],
        vertices=vertices,
        vol=vol,
        bbox=bbox)

    lmbda, mu = homo.get_lambda_and_mu()
    beta = mu/2
    alpha = lmbda/4


    W_skel = beta * (I_C_CM - 2 - 2 * sympy.ln(J_CM)) + alpha * (J_CM**2 - 1 - 2 * sympy.ln(J_CM)) 
    Sigma_CM = 2*sympy.diff(W_skel, C_CM) - p * J_CM * C_CM.inv()

    Sigma = Sigma_CM.subs(list(zip(C_CM, C))).doit().as_explicit() 
    sigma = F * Sigma * F.T / J 
    sigma = sigma.subs(alpha_x, epsilon + 1).subs(alpha_y, epsilon + 1)


    Ter_Sigma = 2*sympy.diff(W_skel, C_CM)      
    Ter_Sigma = Ter_Sigma.subs(list(zip(C_CM, C))).doit().as_explicit() 
    Ter_Sigma = Ter_Sigma.subs(alpha_x, epsilon + 1).subs(alpha_y, epsilon + 1)

    return sigma, Ter_Sigma

## Global response of the micromechanical model is a function of:
* material parameters
* $\epsilon_{xx}, \epsilon_{yy}$ $\longrightarrow$ macroscopic strain
* If no macroscopic strain is applied, `Macroscopic_strain` = <span style="color:lightblue">None</span>
* sigma_bar is the applied stress, otherwise:
\begin{equation}
    \sigma = \begin{bmatrix}
                0 & 0\\
                0 & 0
             \end{bmatrix}
\end{equation}

<!-- * $\gamma$ is the surface tension coefficient, otherwise it is equal to zero -->

In [ ]:
def global_response(mesh, mat_params, eps_xx, eps_yy, pf, Macroscopic_strain, sigma_bar):


    dim = 2
    bcs = "pbc"
    step_params = {"dt_ini":1e-1, "dt_min":1e-3}


    res_folder = sys.argv[0][:-3]
    res_basename  = sys.argv[0][:-3]

    res_basename = res_folder+"/"+res_basename
    verbose=1

    ################################################################### Mesh ###


    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    vol = (xmax - xmin)*(ymax - ymin)
    dV = dolfin.Measure("dx",domain=mesh)

    vertices = numpy.array([[xmin, ymin],
                         [xmax, ymin],
                         [xmax, ymax],
                         [xmin, ymax]])

    tol = 1E-8  
    vv = vertices
    a1 = vv[1,:]-vv[0,:] # first vector generating periodicity
    a2 = vv[3,:]-vv[0,:] # second vector generating periodicity
    # check if UC vertices form indeed a parallelogram
    assert numpy.linalg.norm(vv[2, :]-vv[3, :] - a1) <= tol
    assert numpy.linalg.norm(vv[2, :]-vv[1, :] - a2) <= tol
    bbox = [xmin, xmax, ymin, ymax]

    ################################################## Subdomains & Measures ###

    
    xmin_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmin, tol=tol)
    xmax_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmax, tol=tol)
    ymin_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymin, tol=tol)
    ymax_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymax, tol=tol)

    xmin_id = 1
    xmax_id = 2
    ymin_id = 3
    ymax_id = 4


    boundaries_mf = dolfin.MeshFunction("size_t", mesh, mesh.topology().dim()-1) # MG20180418: size_t looks like unisgned int, but more robust wrt architecture and os
    boundaries_mf.set_all(0)

    xmin_sd.mark(boundaries_mf, xmin_id)
    xmax_sd.mark(boundaries_mf, xmax_id)
    ymin_sd.mark(boundaries_mf, ymin_id)
    ymax_sd.mark(boundaries_mf, ymax_id)

    ################################################################ Problem ###

    problem = dmech.MicroPoroHyperelasticityProblem(
        mesh=mesh,
        mesh_bbox=bbox,
        vertices=vertices,
        boundaries_mf=boundaries_mf,
        displacement_perturbation_degree=2,
        quadrature_degree=3,
        solid_behavior=mat_params,
        bcs=bcs)

    ################################################################ Loading ###

    Deltat = step_params.get("Deltat", 1.)
    dt_ini = step_params.get("dt_ini", 1.)
    dt_min = step_params.get("dt_min", 1.)
    dt_max = step_params.get("dt_max", 1.)
    k_step = problem.add_step(
        Deltat=Deltat,
        dt_ini=dt_ini,
        dt_min=dt_min,
        dt_max=dt_max)


    problem.add_surface_pressure_loading_operator(
        measure=problem.dS(0),
        P_ini=0., P_fin=pf,
        k_step=k_step)

    for k in range(dim):
        for l in range (dim):
            if (sigma_bar[k][l] is not None):
                problem.add_macroscopic_stress_component_constraint_operator(
                    i=k, j=l,
                    sigma_bar_ij_ini=0.0, sigma_bar_ij_fin=sigma_bar[k][l],
                    pf_ini=0.0, pf_fin=pf,
                    k_step=k_step)

    if (Macroscopic_strain is not None):
        problem.add_macroscopic_stretch_component_penalty_operator(
            i=0, j=0,
            U_bar_ij_ini=0.0, U_bar_ij_fin=eps_xx,
            pen_val=1e6,
            k_step=k_step)
        problem.add_macroscopic_stretch_component_penalty_operator(
            i=1, j=1,
            U_bar_ij_ini=0.0, U_bar_ij_fin=eps_yy,
            pen_val=1e6,
            k_step=k_step)

    ################################################################# Solver ###

    solver = dmech.NonlinearSolver(
        problem=problem,
        parameters={
            "sol_tol":[1e-6]*len(problem.subsols),
            "n_iter_max":32},
        relax_type="constant",
        write_iter=0)

    integrator = dmech.TimeIntegrator(
        problem=problem,
        solver=solver,
        parameters={
            "n_iter_for_accel":4,
            "n_iter_for_decel":16,
            "accel_coeff":2,
            "decel_coeff":2},
        print_out=res_basename*verbose,
        print_sta=res_basename*verbose,
        write_qois=res_basename+"-qois",
        write_qois_limited_precision=1,
        write_sol=res_basename*verbose)

    success = integrator.integrate()
    assert (success),\
        "Integration failed. Aborting."

    integrator.close()
    

    for operator in problem.operators: 
        if hasattr(operator, "material"):
            material = operator.material
            break
    
    Vs0 = problem.mesh_V0
    U_bar = problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar
    E_bar = 0.5*(C_bar - numpy.eye(2))
    v = J_bar * vol
    vs = dolfin.assemble(problem.kinematics.J * problem.dV)
    vf = v - vs
    
    Phi_s = vs/vol


    
    sigma_tot_xx = float((dolfin.assemble(material.sigma[0,0] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[0,0])/v)
    sigma_tot_yy = float((dolfin.assemble(material.sigma[1,1] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[1,1])/v)

    return sigma_tot_xx


## Macro-micro model comparision


In [ ]:
sigma_bar = [[None, 0.],
             [0., None]]

### Stress-strain comparision micro-Macro for differnent $E_s$

In [ ]:
# Micro model
sigma_glob_Es_1 = []
sigma_glob_Es_2 = []
sigma_glob_Es_3 = []

eps_glob = []

for i in range(11):
    eps = i/20
    eps_glob.append(eps)
    mat_params = {"model":"CGNH", "parameters":{"E":0.5, "nu":nus}}
    sigma_glob_Es_1.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar))
    mat_params = {"model":"CGNH", "parameters":{"E":1, "nu":nus}}
    sigma_glob_Es_2.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar))
    mat_params = {"model":"CGNH", "parameters":{"E":2, "nu":nus}}
    sigma_glob_Es_3.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar))


# Macro model
mat_params = {"model":"CGNH", "parameters":{"E":0.5, "nu":nus}}
sigma_macro_1 = macroscopic_model(mat_params)[0].subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":1, "nu":nus}}
sigma_macro_2 = macroscopic_model(mat_params)[0].subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":2, "nu":nus}}
sigma_macro_3 = macroscopic_model(mat_params)[0].subs(p, 0)


epsilon_vals = numpy.linspace(0, 0.5, 100)

lam = scipy.lambdify(epsilon, sigma_macro_1[0, 0], modules=['numpy'])
sigma_vals_1 = lam(epsilon_vals)

lam = scipy.lambdify(epsilon, sigma_macro_2[0, 0], modules=['numpy'])
sigma_vals_2 = lam(epsilon_vals)

lam = scipy.lambdify(epsilon, sigma_macro_3[0, 0], modules=['numpy'])
sigma_vals_3 = lam(epsilon_vals)

In [ ]:
class MulticolorPatch(object):
    def __init__(self, colors):
        self.colors = colors
        
# define a handler for the MulticolorPatch object
class MulticolorPatchHandler(object):
    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        width, height = handlebox.width, handlebox.height
        patches = []
        for i, c in enumerate(orig_handle.colors):
            patches.append(plt.Rectangle([width/len(orig_handle.colors) * i - handlebox.xdescent, 
                                          -handlebox.ydescent],
                           width / len(orig_handle.colors),
                           height, 
                           facecolor=c, 
                           edgecolor='none'))

        patch = PatchCollection(patches,match_original=True)

        handlebox.add_artist(patch)
        return patch


# ------ choose some colors
colors1 = ['#084594', '#99000D']
colors2 = ['#4292C6', '#EF3B2C']
colors3 = ['#9ECAE1', '#FC9272']

# ------ create a dummy-plot (just to show that it works)
f, ax = plt.subplots()
ax.plot(eps_glob, sigma_glob_Es_1, c=colors1[0])
ax.plot(eps_glob, sigma_glob_Es_2, c=colors2[0])
ax.plot(eps_glob, sigma_glob_Es_3, c=colors3[0])
ax.plot(epsilon_vals, sigma_vals_1, c=colors1[1])
ax.plot(epsilon_vals, sigma_vals_2, c=colors2[1])
ax.plot(epsilon_vals, sigma_vals_3, c=colors3[1])

# ------ get the legend-entries that are already attached to the axis
h, l = ax.get_legend_handles_labels()

# ------ append the multicolor legend patches
h.append(MulticolorPatch(colors1))
l.append(r'$E_s = 0.5~kPa$')

h.append(MulticolorPatch(colors2))
l.append(r'$E_s = 1~kPa$')

h.append(MulticolorPatch(colors3))
l.append(r'$E_s = 2~kPa$')

# ------ create the legend
f.legend(h, l, loc='upper left', 
         handler_map={MulticolorPatch: MulticolorPatchHandler()}, 
         bbox_to_anchor=(.125,.875))

plt.xlim(0, 0.5)
plt.ylim(0, 0.175)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x,~E_y~()$', fontsize=16)
plt.ylabel(r'$\sigma~(kPa)$', fontsize=16)
# plt.savefig('stress_strain_micro_macro_Es.pdf',bbox_inches='tight')
plt.show()

### Stress-strain comparision micro-Macro for differnent $\nu_s$

In [ ]:
sigma_glob_nu_01 = []
sigma_glob_nu_02 = []
sigma_glob_nu_03 = []

eps_glob = []

for i in range(11):
    eps = i/20
    eps_glob.append(eps)
    mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.1}}
    sigma_glob_nu_01.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar))
    mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.2}}
    sigma_glob_nu_02.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar))
    mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.4}}
    sigma_glob_nu_03.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar))

mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.1}}
sigma_macro_1 = macroscopic_model(mat_params)[0].subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.2}}
sigma_macro_2 = macroscopic_model(mat_params)[0].subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.4}}
sigma_macro_3 = macroscopic_model(mat_params)[0].subs(p, 0)

epsilon_vals = numpy.linspace(0, 0.5, 100)

lam = scipy.lambdify(epsilon, sigma_macro_1[0, 0], modules=['numpy'])
sigma_macro_1 = lam(epsilon_vals)

lam = scipy.lambdify(epsilon, sigma_macro_2[0, 0], modules=['numpy'])
sigma_macro_2 = lam(epsilon_vals)

lam = scipy.lambdify(epsilon, sigma_macro_3[0, 0], modules=['numpy'])
sigma_macro_3 = lam(epsilon_vals)

In [ ]:
f, ax = plt.subplots()
ax.plot(eps_glob, sigma_glob_nu_01, c=colors1[0])
ax.plot(eps_glob, sigma_glob_nu_02, c=colors2[0])
ax.plot(eps_glob, sigma_glob_nu_03, c=colors3[0])
ax.plot(epsilon_vals, sigma_macro_1, c=colors1[1])
ax.plot(epsilon_vals, sigma_macro_2, c=colors2[1])
ax.plot(epsilon_vals, sigma_macro_3, c=colors3[1])

# ------ get the legend-entries that are already attached to the axis
h, l = ax.get_legend_handles_labels()

# ------ append the multicolor legend patches
h.append(MulticolorPatch(colors1))
l.append(r'$\nu_s = 0.1$')

h.append(MulticolorPatch(colors2))
l.append(r'$\nu_s = 0.2$')

h.append(MulticolorPatch(colors3))
l.append(r'$\nu_s = 0.4$')

# ------ create the legend
f.legend(h, l, loc='upper left', 
         handler_map={MulticolorPatch: MulticolorPatchHandler()}, 
         bbox_to_anchor=(.125,.875))


plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x,~E_y~()$', fontsize=16)
plt.ylabel(r'$\sigma~(kPa)$', fontsize=16)
plt.xlim(0, 0.5)
plt.ylim(0, 0.175)
# plt.savefig('stress_strain_micro_macro_nus.pdf',bbox_inches='tight')
plt.show()